In [1]:
import pandas as pd
import numpy as np
from Lib import Algen_lib as lib

# Load Data

In [2]:
#Data Latih
dfA = pd.read_excel('data/sms_clean_norm_+_new_data_3.xlsx')
df =dfA[100:] 
corpus = df.sms.tolist()
label = df.label.tolist()
len(dfA)

1143

### Pembagian Data Uji

In [3]:
#Data Uji
df =dfA[:100] 
corpusU = df.sms.tolist()
labelU = df.label.tolist()

In [4]:
# df =dfA[100:] 
corpusA = dfA.sms.tolist()
labelA = dfA.label.tolist()

# Ekstraksi Fitur

In [5]:
fitur = lib.feature_extraction(corpus)

# Seleksi Fitur Menggunaka Algoritma Genetika

In [7]:
jumlah_populasi = 500
jumlah_point = 1000 #crossover
prob_mutasi = 0.2 #probabilitas permutasi

K=12

metode = "tfidf"
alpha = 1

target = 0.99 #target akurasi

fitur = lib.feature_extraction(corpus)
panjang_fitur = len(fitur)

features_1_bin = [1 for i in range(panjang_fitur)]
pop = lib.create_population(jumlah_populasi-1, panjang_fitur)
pop.append(features_1_bin)

fns_1 = lib.fitness_kf(corpusA, labelA, features_1_bin, fitur, alpha = alpha, metode = metode, K=K)
print("fitness all features:",fns_1)

fitnes_pop = list()
temp_features_fitness = list()
for features_bin in pop:
    fns = lib.fitness_kf(corpusA, labelA, features_bin, fitur, alpha = alpha, metode = metode, K=K)
    fitnes_pop.append(fns)
    temp_features_fitness.append([fns, features_bin])

pop_used = dict()
popp_0 = list(pop)
for p, fit in zip(popp_0, fitnes_pop):
    bin_str = lib.int_to_str(p) 
    pop_used.update({bin_str:fit})

# print("")
    
# print(pop[0])

best_fitur_list = list()
best_fitness_list = list()
best_generasi_list = list()

# save_aja = list()

gen=0
while True:
    gen+=1
    print("-"*25)
    print("Generasi ke",gen , "[Terbaik]")
    rw = lib.get_roulette_wheel_(fitnes_pop, inc=jumlah_populasi)
    parents = lib.select_parents(rw)

    best_fitur_list_per_generasi = list()
    best_fitness_list_per_generasi = list()
    for p in parents:
        best_generasi_list.append(gen)
        family_fitness=list()
        family_fitur=list()

        mama_index = p[0]
        papa_index = p[1]
        
        m4ms = fitnes_pop[mama_index]
        family_fitness.append(m4ms)
        m4mr = list(pop[mama_index])
        family_fitur.append(m4mr)
        
        p4ps = fitnes_pop[papa_index]
        family_fitness.append(p4ps)
        p4pr = list(pop[papa_index])
        family_fitur.append(p4pr)
        
        #crossover dan mutasi
        bin_mama = list(pop[mama_index])
        bin_papa = list(pop[papa_index])
        anak_binary = lib.crossover(bin_mama, bin_papa, panjang_fitur,  jumlah_titik = jumlah_point, prob_mutasi = prob_mutasi)
        
        #mencari fitness untuk child
        a1 = anak_binary[0]
        a2 = anak_binary[1]
#         print(a2)
        
        if sum(a1)<=0:
            a1[-1]=1
            a1[-2]=1
        if sum(a2)<=0:
            a2[-1]=1
            a2[-2]=1

        str_a1 = lib.int_to_str(a1)
        if str_a1 in pop_used:
            family_fitur.append(a1)
            family_fitness.append(pop_used[str_a1])  
        else:
            family_fitur.append(a1)
            fitness_anak_1 = lib.fitness_kf(corpusA, labelA, a1, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_1)
            pop_used.update({str_a1:fitness_anak_1})
            
        str_a2 = lib.int_to_str(a2)
        if str_a2 in pop_used:
            family_fitur.append(a2)
            family_fitness.append(pop_used[str_a2])
        else:
            family_fitur.append(a2)
            fitness_anak_2 = lib.fitness_kf(corpusA, labelA, a2, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_2)
            pop_used.update({str_a2:fitness_anak_2})

        
        #mencari fitness terbaik untuk satu keluarga
        best_family_fitness = max(family_fitness)
        bf_index = family_fitness.index(best_family_fitness)
        best_family_fitur = family_fitur[bf_index]

        best_fitness_list.append(best_family_fitness)
        best_fitur_list.append(best_family_fitur)

        best_fitness_list_per_generasi.append(best_family_fitness)
        best_fitur_list_per_generasi.append(best_family_fitur)

#         print("   ",best_family_fitness)
        print("   ",lib.tampil_finess(best_family_fitness),"|",str(sum(best_family_fitur))+"|"+str(len(best_family_fitur)),"|",gen)

    pop = list(best_fitur_list_per_generasi)
#     pop.append(features_1_bin)
    fitnes_pop = list(best_fitness_list_per_generasi)
#     fitnes_pop.append(fns_1)

    if len(pop)<2 or max(best_fitness_list)>=target:
        print("="*70)
        print("Best",max(best_fitness_list))
#         print("Generasi ke-",best_generasi_list[best_fitness_list.index(max(best_fitness_list))])
        good_fitur = best_fitur_list[best_fitness_list.index(max(best_fitness_list))]
        best_fitness_list.index(max(best_fitness_list))
        print('jumlah fitur     ', sum(good_fitur))
        print('jumlah fitur asli', panjang_fitur)
        print('good_fitur')
        break

fitness all features: 0.9352796052631581
-------------------------
Generasi ke 1 [Terbaik]
    0.9133954678362572   | 2133|4144 | 1
    0.9116593567251462   | 2092|4144 | 1
    0.9177540204678363   | 2128|4144 | 1
    0.9159722222222223   | 2019|4144 | 1
    0.9098592836257309   | 2051|4144 | 1
    0.9116045321637428   | 2062|4144 | 1
    0.9054824561403509   | 1983|4144 | 1
    0.9220851608187134   | 2042|4144 | 1
    0.9020193713450294   | 2086|4144 | 1
    0.9133954678362574   | 2140|4144 | 1
    0.9203581871345029   | 2090|4144 | 1
    0.9142817982456141   | 2053|4144 | 1
    0.9107547514619884   | 2082|4144 | 1
    0.9081140350877193   | 2103|4144 | 1
    0.9204038742690058   | 2113|4144 | 1
    0.9247258771929826   | 2086|4144 | 1
    0.9143183479532163   | 1991|4144 | 1
    0.9160361842105263   | 2103|4144 | 1
    0.9142269736842107   | 2063|4144 | 1
    0.919544956140351    | 2032|4144 | 1
    0.9204312865497076   | 2083|4144 | 1
    0.9151407163742689   | 2079|4144 | 1
    0.9

    0.915140716374269    | 2045|4144 | 1
    0.9159904970760233   | 2111|4144 | 1
    0.9150767543859649   | 2026|4144 | 1
    0.9168768274853801   | 2081|4144 | 1
    0.9124817251461989   | 2056|4144 | 1
    0.9133589181286551   | 2102|4144 | 1
    0.9168951023391814   | 2069|4144 | 1
    0.907282529239766    | 2044|4144 | 1
    0.9160087719298246   | 2082|4144 | 1
    0.9125               | 2073|4144 | 1
    0.9134320175438596   | 2116|4144 | 1
    0.9081323099415205   | 2074|4144 | 1
    0.918622076023392    | 2030|4144 | 1
    0.915140716374269    | 2078|4144 | 1
    0.9203947368421052   | 2035|4144 | 1
    0.9168859649122806   | 2120|4144 | 1
    0.9177266081871344   | 2042|4144 | 1
    0.9116410818713451   | 2077|4144 | 1
    0.9116502192982455   | 2049|4144 | 1
    0.9107090643274853   | 2107|4144 | 1
    0.9160270467836257   | 2074|4144 | 1
    0.9203581871345029   | 2062|4144 | 1
    0.9186586257309942   | 2064|4144 | 1
    0.9159904970760232   | 2077|4144 | 1
    0.9186220760

    0.9220851608187134   | 2042|4144 | 3
    0.9221034356725147   | 2069|4144 | 3
    0.9203947368421052   | 2035|4144 | 3
    0.9169133771929826   | 2056|4144 | 3
    0.919544956140351    | 2032|4144 | 3
    0.9151315789473683   | 2072|4144 | 3
    0.9204312865497076   | 2083|4144 | 3
    0.9159722222222223   | 2019|4144 | 3
    0.9186129385964912   | 2082|4144 | 3
    0.916858552631579    | 2090|4144 | 3
    0.9247532894736844   | 2063|4144 | 3
    0.9195084064327484   | 2086|4144 | 3
    0.9168585526315788   | 2079|4144 | 3
    0.922139985380117    | 2063|4144 | 3
    0.919499269005848    | 2113|4144 | 3
    0.9186403508771929   | 2080|4144 | 3
    0.9264711257309942   | 2014|4144 | 3
    0.9229897660818714   | 2095|4144 | 3
    0.916858552631579    | 2059|4144 | 3
    0.9168951023391813   | 2079|4144 | 3
    0.9247989766081871   | 2110|4144 | 3
    0.9204038742690058   | 2113|4144 | 3
    0.9230445906432747   | 2059|4144 | 3
    0.9160270467836257   | 2103|4144 | 3
    0.9177722953

In [8]:
#Testing
hasil_seleksi_fitur = good_fitur
lib.fitness_(corpus,label, corpusU,labelU, hasil_seleksi_fitur, fitur, alpha = alpha, metode = metode)

0.92

In [8]:
fitur_pilihan = np.array(fitur)[lib.get_index(good_fitur)]
print(len(fitur_pilihan))
fitur_pilihan

4144


array(['0', '00', '000', ..., 'zalora', 'zarkasi', 'zona'], dtype='<U26')

In [62]:
# skor_all